In [1]:
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np
import os
from statistics import mean, stdev

fp = "G:\\My Drive\\Darby Work\\Ytsma and Dyar 2021 (LOD paper)\\"

### Braga method

In [2]:
# define blank/noise regions in ChemLIBS and ChemCam spectra
blank_braga = pd.read_csv(fp + "figures\\braga_noise_regions.csv") 

# import all spectra
dp = fp + 'datasets//'
cc_mars = pd.read_csv(dp+'CC_norm3_spectra.csv')
cl_mars = pd.read_csv(dp+'CL_Mars_norm3_spectra_matched.csv')
cl_earth = pd.read_csv(dp+'CL_Earth_norm3_spectra_matched.csv')
cl_vac = pd.read_csv(dp+'CL_Vacuum_norm3_spectra_matched.csv')

In [3]:
# filter regions
df_list = [cc_mars, cl_earth, cl_mars, cl_vac]

for df in tqdm(df_list):
    for row in tqdm(df.index, leave=False):
        nm = df['wave'][row]
        # remove rows below first region
        if nm < blank_braga['start'][0]:
            df.drop(row, axis = 'index', inplace=True)
        # remove rows after last region
        elif nm > blank_braga['stop'][len(blank_braga)-1]:
            df.drop(row, axis = 'index', inplace=True)
        # remove rows between the regions
        for region in range(len(blank_braga)-1):
            if (nm > blank_braga['stop'][region]) & (nm < blank_braga['start'][region+1]):
                df.drop(row, axis = 'index', inplace=True)

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/5483 [00:00<?, ?it/s]

  0%|          | 0/5483 [00:00<?, ?it/s]

  0%|          | 0/5483 [00:00<?, ?it/s]

  0%|          | 0/5483 [00:00<?, ?it/s]

In [4]:
# redefine with updated dfs
df_list = [cc_mars,cl_earth, cl_mars, cl_vac]
inst_list = ['CC', 'CL', 'CL', 'CL']
atm_list = ["Mars", "Earth", "Mars", "Vacuum"]

In [5]:
# summarise stdev across each row, take average
sens_list = []
sd_list = []
for df in df_list:
    sens = round(df.set_index('wave').std(axis=1).mean(),9)
    sd = round(df.set_index('wave').std(axis=1).std(),9)
    sens_list.append(sens)
    sd_list.append(sd)

# make dataframe
braga_sensitivities = pd.DataFrame({
    "instrument" : inst_list,
    "atmosphere" : atm_list,
    "sensitivity" : sens_list,
    'stdev' : sd_list
})

### Metals method
#### ChemLIBS

In [6]:
sp = dp + 'sensitivity//'
earth_meta = pd.read_csv(sp+'metals_Earth_meta.csv')
mars_meta = pd.read_csv(sp+'metals_Mars_meta.csv')
vac_meta = pd.read_csv(sp+'metals_Vacuum_meta.csv')

earth_norm3 = pd.read_csv(sp+'metals_Earth_norm3.csv')
mars_norm3 = pd.read_csv(sp+'metals_Mars_norm3.csv')
vac_norm3 = pd.read_csv(sp+'metals_Vacuum_norm3.csv')

In [7]:
df_list = [earth_norm3, mars_norm3, vac_norm3]
atm_list = ['Earth', 'Mars', 'Vacuum']
elems = ['AL', 'CR', 'CU', 'FE', 'SN', 'V']

In [8]:
count = 0
sens_list = []
sd_list = []

for df in df_list:
    
    atm = atm_list[count]
    count+=1
    
    if atm == 'Earth':
        meta = earth_meta
    elif atm == 'Mars':
        meta = mars_meta
    elif atm == 'Vacuum':
        meta = vac_meta
    
    s_list = []
    
    for element in elems:
        
        spec_names = list(meta[meta['Sample Name'] == str(element+'METAL')]['pkey'])
        spectra = df[spec_names]
    
        # calculate sensitivity
        sensitivity = round(spectra.std(axis=1).mean(),9)
        s_list.append(sensitivity)
        
    # average results from all elements
    sens = mean(s_list)
    sd = stdev(s_list)
    sens_list.append(sens)
    sd_list.append(sd)

# make dataframe of all results
metal_sens = pd.DataFrame({
    "instrument" : ['CL'] * 3,
    "atmosphere" : atm_list,
    "sensitivity" : sens_list,
    'stdev' : sd_list
})

#### LANL

In [9]:
norm_25 = pd.read_csv(sp+'25_tio2_spectra_norm3.csv')
norm_50 = pd.read_csv(sp+'50_tio2_spectra_norm3.csv')
norm_75 = pd.read_csv(sp+'75_tio2_spectra_norm3.csv')

In [10]:
sens_list = []
for df in [norm_25, norm_50, norm_75]:
    sens = round(df.set_index('wave').std(axis=1).mean(),9)
    sens_list.append(sens)
sens = mean(sens_list)
sd = stdev(sens_list)

metal_sens.loc[3] = ['CC', 'Mars', sens, sd]

Combine results

In [11]:
# merge
metal_sens['method'] = "metals"
braga_sensitivities['method'] = "braga"

sensitivities = pd.concat([metal_sens, braga_sensitivities]).reset_index(drop=True)

In [12]:
sensitivities.to_csv(fp+'instrument_sensitivities.csv', index=False)